In [2]:

import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
# grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10)
grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10, n_jobs=-1)
grid_search.fit(digits.data, digits.target)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=300, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-06, 2.59294380e-06, 6.72335754e-06, 1.74332882e-05,
       4.52035366e-05, 1.17210230e-04, 3.03919538e-04, 7.88046282e-04,
       2.04335972e-03, 5.29831691e-03, 1.37382380e-02, 3.56224789e-02,
       9.23670857e-02, 2.39502662e-01, 6.21016942e-01, 1.61026203e+00,
       4.17531894e+00, 1....
       2.80721620e+07, 1.00000000e+08]),
                                        'tol': array([0.0001    , 0.0001269 , 0.00016103, 0.00020434, 0.00025929,
       0.00032903, 0.00041753, 0.00052983, 0.00067234, 0.00085317,
       0.00108264, 0.00137382, 0.00174333, 0.00221222, 0.00280722,
       0.00356225, 0.00452035, 0.00573615, 0.00727895, 0.00923671,
       0.01172102, 0.01487352, 0.01887392, 0.02395027, 0.03039195,
       0.0385662 , 0.04893901, 0.06210169, 0.07880463, 0.1       ])},
                   verbose=10)

In [7]:
# import ray
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from ray.util.joblib import register_ray
from sklearn.model_selection import RandomizedSearchCV

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}

# En général, il est plus efficace de ne PAS spécifier n_jobs dans l'objet RandomizedSearchCV lorsqu'on utilise Ray comme backend via joblib. 
# Cela permet à Ray de gérer toute la parallélisation, évitant ainsi les conflits ou redondances dans l'utilisation des ressources.
model = SVC(kernel='rbf')
grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300)
# grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10, n_jobs=-1)
# grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10, n_jobs=1)

# ray.init()
register_ray()
with joblib.parallel_backend('ray'):
    grid_search.fit(digits.data, digits.target)
# ray.shutdown()

In [11]:
import ray
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_digits
# from ray.util.joblib import register_ray
from sklearn.model_selection import RandomizedSearchCV

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}

model = SVC(kernel='rbf')
grid_search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300)

# ray.shutdown()
ray.init(ignore_reinit_error=True)
register_ray()
with joblib.parallel_backend('ray'):
    grid_search.fit(digits.data, digits.target)
ray.shutdown()

2024-06-28 07:41:44,761	INFO worker.py:1604 -- Calling ray.init() again after it has already been called.
